# Exercise 2 - One-vs-all MNIST
## Imports

In [29]:
import numpy as np

# So that changes to the a2 model are reflected here.
import a3
import importlib
importlib.reload(a3)
import a3
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_digits
from matplotlib import pyplot as plt
import pickle

## Functions

In [30]:
def predict(clfs, test):
    preds = []
    for i in range(len(clfs)):
        print(clfs[i].predict(test))
        if clfs[i].predict(test) == 1:
            preds.append(i)

def save_pickle(file_path, data):
    file = open(file_path, 'ab')
    pickle.dump(data, file)
    file.close()

def load_pickle(file_path):
    file = open(file_path, 'rb')
    data = pickle.load(file)
    file.close()
    return data

## Part 1 - Load & trim MNIST dataset

In [31]:
X_train, y_train, X_test, y_test = a3.mnist()

# Trim data.
train_size = 1000
test_size = 1000
X_train, y_train = X_train[:train_size, :], y_train[:train_size, :]
X_test, y_test = X_test[:test_size, :], y_test[:test_size, :]

## Part 2 - Train or load SVMs

In [32]:
# True to load from disk, False to train models again.
load_grids = True

if load_grids:
    grids = load_pickle('pickles/grids.pickle')
else:
    params = {
        'C' : [0.1, 1, 10, 100],
        'gamma' : [1, 0.1, 0.01, 'scale', 'auto'],
    }

    # Perform grid search
    grids = []
    for col in range(y_train.shape[1]):
        print("Training model for column", col)
        grid = GridSearchCV(SVC(), params, n_jobs=-1)
        grid.fit(X_train, y_train[:, col])
        grids.append(grid)

    save_pickle('pickles/grids.pickle', grids)

# Print classifier scores
print("Cross validation accuracy for each digit response classifier:")
for i in range(len(grids)):
    score = round(grids[i].best_score_, 3)
    print(f"{i}: {score}%")

clfs = [grid.best_estimator_ for grid in grids]

Accuracy for each digit response classifier:
0: 0.991%
1: 0.992%
2: 0.977%
3: 0.983%
4: 0.979%
5: 0.979%
6: 0.985%
7: 0.975%
8: 0.971%
9: 0.971%
